### NAMED ENTITY RECOGNITION

In [ ]:
!pip install transformers

In [ ]:
!pip install simpletransformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
data =  pd.read_csv("/content/drive/MyDrive/ner_dataset.csv", encoding = 'latin1')

In [ ]:
data.head(30)

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O
5,NaN,through,IN,O
6,NaN,London,NNP,B-geo
7,NaN,to,TO,O
8,NaN,protest,VB,O
9,NaN,the,DT,O


In [ ]:
data = data.fillna(method='ffill')

In [ ]:
data.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O


In [ ]:
from importlib import import_module
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
data["Sentence #"] = LabelEncoder().fit_transform(data["Sentence #"])

In [ ]:
data.head(15)

,Sentence #,Word,POS,Tag
0,0,Thousands,NNS,O
1,0,of,IN,O
2,0,demonstrators,NNS,O
3,0,have,VBP,O
4,0,marched,VBN,O
5,0,through,IN,O
6,0,London,NNP,B-geo
7,0,to,TO,O
8,0,protest,VB,O
9,0,the,DT,O


In [ ]:
data.rename(columns={"Sentence #":"sentence_id","Word":"words","Tag":"labels"}, inplace =True)
#true is for the purpose renaming the dataset completely

In [ ]:
data["labels"] = data["labels"].str.upper()

In [ ]:
data.head(30)

,sentence_id,words,POS,labels
0,0,Thousands,NNS,O
1,0,of,IN,O
2,0,demonstrators,NNS,O
3,0,have,VBP,O
4,0,marched,VBN,O
5,0,through,IN,O
6,0,London,NNP,B-GEO
7,0,to,TO,O
8,0,protest,VB,O
9,0,the,DT,O


In [ ]:
data.shape

(1048575, 4)

In [ ]:
X= data[["sentence_id","words"]]   #DEPENDENT VARIABLE (This is used for labelling)
Y =data["labels"]  #TARGET VARIABLE(Labelling is the target here)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size =0.2) #test percent if it is high the accuracy will be low


In [ ]:
train_data = pd.DataFrame({"sentence_id":x_train["sentence_id"],"words":x_train["words"],"labels":y_train})
test_data = pd.DataFrame({"sentence_id":x_test["sentence_id"],"words":x_test["words"],"labels":y_test})
#Using dataframe () we are structing the test and train data here like returning the data of str,int and float like that

In [ ]:
train_data.shape

(838860, 3)

In [ ]:
test_data.shape

(209715, 3)

In [ ]:
train_data.head(10)

,sentence_id,words,labels
516397,15130,the,O
902631,34697,50,O
749263,26949,was,O
975865,38474,NASA,B-ORG
950835,37175,seized,O
13114,43416,unclear,O
508267,14708,Nobel,B-ORG
366130,7499,with,O
457631,12138,was,O
80676,29679,plane,O


### Model Training

In [ ]:
from simpletransformers.ner import NERModel, NERArgs
#In Nermodel we are importing the NERArgs model

In [ ]:
label = data["labels"].unique().tolist()
label
#B is begging of the sentence
#I is inside the sentence
#O is outside the sentence

#For eg B-GEO is one of the entity labelling tags


['O',
 'B-GEO',
 'B-GPE',
 'B-PER',
 'I-GEO',
 'B-ORG',
 'I-ORG',
 'B-TIM',
 'B-ART',
 'I-ART',
 'I-PER',
 'I-GPE',
 'I-TIM',
 'B-NAT',
 'B-EVE',
 'I-EVE',
 'I-NAT']

In [ ]:
args = NERArgs()
args.num_train_epochs = 1  #In an algorithm(BERT) if a dataset goes in and comes out(cycle) after it is been trained is called as epoch
args.learning_rate = 1e-4  #In which speed its should be learnt, it is mainly used for the purpose of tuning parameter in an optimisation algorithm (to reduce the losss rate)
args.overwrite_output_dir =True #For output storing
args.train_batch_size = 32 #In one epoch 32 batches will be there for trainig and evaluation
args.eval_batch_size = 32 # In one epoch 32 batches will be there for trainig and evaluation

In [ ]:
!which nvidia-smi

/opt/bin/nvidia-smi


In [ ]:
model = NERModel('bert', 'bert-base-cased',labels=label,args = args)
#BERT IS THE ONE IN WHICH WE ARE TRAINING


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [ ]:
model.train_model(train_data,eval_data = test_data,acc=accuracy_score)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


  0%|          | 0/2 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 1 of 1:   0%|          | 0/1499 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:143: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


(1499, 0.19791892140586825)

In [ ]:
results,model_outputs,pred_list =model.eval_model(test_data)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


  0%|          | 0/3 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1461 [00:00<?, ?it/s]

In [ ]:
results
#eval loss , alll theses are machine learning terms
#f_1 score is the overall performance of the model
#precison is true predictions (positive like it is tagged the label correctly)
# recall is the proportion of the true preditions in the overall dataset

{'eval_loss': 0.17231860112534003,
 'precision': 0.8285424534899986,
 'recall': 0.7563848806027327,
 'f1_score': 0.7908210947930573}

In [ ]:
prediction, model_output =model.predict([" Ms J. K. is an 83 year old retired nurse with a long history of hypertension that was previously well controlled on diuretic therapy. She was first admitted to CPMC in 1995 when she presented with a complaint of intermittent midsternal chest pain."])

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
prediction

[[{'Ms': 'B-PER'},
  {'J.': 'I-PER'},
  {'K.': 'I-PER'},
  {'is': 'O'},
  {'an': 'O'},
  {'83': 'O'},
  {'year': 'O'},
  {'old': 'O'},
  {'retired': 'O'},
  {'nurse': 'O'},
  {'with': 'O'},
  {'a': 'O'},
  {'long': 'O'},
  {'history': 'O'},
  {'of': 'O'},
  {'hypertension': 'O'},
  {'that': 'O'},
  {'was': 'O'},
  {'previously': 'O'},
  {'well': 'O'},
  {'controlled': 'O'},
  {'on': 'O'},
  {'diuretic': 'O'},
  {'therapy.': 'O'},
  {'She': 'O'},
  {'was': 'O'},
  {'first': 'O'},
  {'admitted': 'O'},
  {'to': 'O'},
  {'CPMC': 'B-ORG'},
  {'in': 'O'},
  {'1995': 'B-TIM'},
  {'when': 'O'},
  {'she': 'O'},
  {'presented': 'O'},
  {'with': 'O'},
  {'a': 'O'},
  {'complaint': 'O'},
  {'of': 'O'},
  {'intermittent': 'O'},
  {'midsternal': 'O'},
  {'chest': 'O'},
  {'pain.': 'O'}]]